In [34]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import sys
import pandas as pd
from typing import *
from enum import Enum
import os
import numpy as np


# Now you can import from the parent directory



class GeneralConstants(Enum):
    """
    Holds constants for calculations and conversions
    1. covalent radii from Alvarez (2008) DOI: 10.1039/b801115j
    2. atomic numbers
    2. atomic weights
    """
    COVALENT_RADII= {
            'H': 0.31, 'He': 0.28, 'Li': 1.28,
            'Be': 0.96, 'B': 0.84, 'C': 0.76, 
            'N': 0.71, 'O': 0.66, 'F': 0.57, 'Ne': 0.58,
            'Na': 1.66, 'Mg': 1.41, 'Al': 1.21, 'Si': 1.11, 
            'P': 1.07, 'S': 1.05, 'Cl': 1.02, 'Ar': 1.06,
            'K': 2.03, 'Ca': 1.76, 'Sc': 1.70, 'Ti': 1.60, 
            'V': 1.53, 'Cr': 1.39, 'Mn': 1.61, 'Fe': 1.52, 
            'Co': 1.50, 'Ni': 1.24, 'Cu': 1.32, 'Zn': 1.22, 
            'Ga': 1.22, 'Ge': 1.20, 'As': 1.19, 'Se': 1.20, 
            'Br': 1.20, 'Kr': 1.16, 'Rb': 2.20, 'Sr': 1.95,
            'Y': 1.90, 'Zr': 1.75, 'Nb': 1.64, 'Mo': 1.54,
            'Tc': 1.47, 'Ru': 1.46, 'Rh': 1.42, 'Pd': 1.39,
            'Ag': 1.45, 'Cd': 1.44, 'In': 1.42, 'Sn': 1.39,
            'Sb': 1.39, 'Te': 1.38, 'I': 1.39, 'Xe': 1.40,
            'Cs': 2.44, 'Ba': 2.15, 'La': 2.07, 'Ce': 2.04,
            'Pr': 2.03, 'Nd': 2.01, 'Pm': 1.99, 'Sm': 1.98,
            'Eu': 1.98, 'Gd': 1.96, 'Tb': 1.94, 'Dy': 1.92,
            'Ho': 1.92, 'Er': 1.89, 'Tm': 1.90, 'Yb': 1.87,
            'Lu': 1.87, 'Hf': 1.75, 'Ta': 1.70, 'W': 1.62,
            'Re': 1.51, 'Os': 1.44, 'Ir': 1.41, 'Pt': 1.36,
            'Au': 1.36, 'Hg': 1.32, 'Tl': 1.45, 'Pb': 1.46,
            'Bi': 1.48, 'Po': 1.40, 'At': 1.50, 'Rn': 1.50, 
            'Fr': 2.60, 'Ra': 2.21, 'Ac': 2.15, 'Th': 2.06,
            'Pa': 2.00, 'U': 1.96, 'Np': 1.90, 'Pu': 1.87,
            'Am': 1.80, 'Cm': 1.69
    }
    BONDI_RADII={
        'H': 1.10, 'C': 1.70, 'F': 1.47,
        'S': 1.80, 'B': 1.92, 'I': 1.98, 
        'N': 1.55, 'O': 1.52, 'Co': 2.00, 
        'Br': 1.83, 'Si': 2.10,'Ni': 2.00,
        'P': 1.80, 'Cl': 1.75, 
    }

def flatten_list(nested_list_arg: List[list]) -> List:
    """
    Flatten a nested list.
    turn [[1,2],[3,4]] to [1,2,3,4]
    """
    flat_list=[item for sublist in nested_list_arg for item in sublist]
    return flat_list


def plot_interactions(xyz_df,color):
    """Creates a 3D plot of the molecule"""

    atomic_radii = GeneralConstants.COVALENT_RADII.value
    cpk_colors = dict(C='black', F='green', H='white', N='blue', O='red', P='orange', S='yellow', Cl='green', Br='brown', I='purple', Ni='blue', Fe='red', Cu='orange', Zn='yellow', Ag='grey', Au='gold',Si='grey', B='pink',Pd='green')

    # if molecule_name not in train_df.molecule_name.unique():
    #     print(f'Molecule "{molecule_name}" is not in the training set!')
    #     return
    #

    coordinates = np.array(xyz_df[['x', 'y', 'z']].values,dtype=float)
    x_coordinates = coordinates[:, 0]
    y_coordinates = coordinates[:, 1]
    z_coordinates = coordinates[:, 2]
    atoms = ((xyz_df['atom'].astype(str)).values.tolist())
    atom_ids = xyz_df.index.tolist()
    try:
        radii = [atomic_radii[atom] for atom in atoms]
    except TypeError:
        atoms = flatten_list(atoms)
        radii = [atomic_radii[atom] for atom in atoms]


    def get_bonds():
        """Generates a set of bonds from atomic cartesian coordinates"""
        ids = np.arange(coordinates.shape[0])
        bonds = dict()
        coordinates_compare, radii_compare, ids_compare = coordinates, radii, ids

        for _ in range(len(ids)):
            coordinates_compare = np.roll(coordinates_compare, -1, axis=0)
            radii_compare = np.roll(radii_compare, -1, axis=0)
            ids_compare = np.roll(ids_compare, -1, axis=0)
            distances = np.linalg.norm(coordinates - coordinates_compare, axis=1)
            bond_distances = (radii + radii_compare) * 1.3
            mask = np.logical_and(distances > 0.1, distances < bond_distances)
            distances = distances.round(2)
            new_bonds = {frozenset([i, j]): dist for i, j, dist in zip(ids[mask], ids_compare[mask], distances[mask])}
            bonds.update(new_bonds)
        return bonds

    def atom_trace():
        """Creates an enhanced atom trace for the plot with increased marker size and border width."""
        # Use the CPK color for each atom
        colors = [cpk_colors[atom] for atom in atoms]
        # Increase marker size, set a bolder border (line) and higher opacity for better visibility
        markers = dict(
            color=colors,
            line=dict(color='black', width=3),  # Bold border for clarity
            size=10,                           # Increased size for better visibility
            symbol='circle',
            opacity=0.9                        # Slightly higher opacity
        )
        trace = go.Scatter3d(
            x=x_coordinates,
            y=y_coordinates,
            z=z_coordinates,
            mode='markers',
            marker=markers,
            text=atoms,
            name='Atoms'
        )
        return trace

    def bond_trace(color):
        """Creates an enhanced bond trace for the plot with thicker lines."""
        # Create an empty trace for bonds with increased line width
        trace = go.Scatter3d(
            x=[], y=[], z=[], 
            hoverinfo='none', 
            mode='lines',
            line=dict(width=5, color=color)  # Thicker lines for bonds
        )
        # Loop over the bond indices and append coordinates, inserting None for line breaks
        for i, j in bonds.keys():
            trace['x'] += (x_coordinates[i], x_coordinates[j], None)
            trace['y'] += (y_coordinates[i], y_coordinates[j], None)
            trace['z'] += (z_coordinates[i], z_coordinates[j], None)
        return trace
    bonds = get_bonds()
    print(bonds)
    
    zipped = zip(atom_ids, x_coordinates, y_coordinates, z_coordinates)
    annotations_id = [dict(text=num+1, x=x, y=y, z=z, showarrow=False, yshift=15, font=dict(color="blue"))
                      for num, x, y, z in zipped]

    annotations_length = []
    for (i, j), dist in bonds.items():
        x_middle, y_middle, z_middle = (coordinates[i] + coordinates[j]) / 2
        annotation = dict(text=dist, x=x_middle, y=y_middle, z=z_middle, showarrow=False, yshift=10)
        annotations_length.append(annotation)

    updatemenus = list([
        dict(buttons=list([
                 dict(label = 'Atom indices',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_id}]),
                 dict(label = 'Bond lengths',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_length}]),
                 dict(label = 'Atom indices & Bond lengths',
                      method = 'relayout',
                      args = [{'scene.annotations': annotations_id + annotations_length}]),
                 dict(label = 'Hide all',
                      method = 'relayout',
                      args = [{'scene.annotations': []}])
                 ]),
                 direction='down',
                 xanchor = 'left',
                 yanchor = 'top'
            ),
    ])
    data = [atom_trace(), bond_trace(color)]
    return data, annotations_id, updatemenus

def choose_conformers_input():
    string=input('Enter the conformers numbers: ')
    conformer_numbers=string.split(' ')
    conformer_numbers=[int(i) for i in conformer_numbers]
    return conformer_numbers


def unite_buttons(buttons_list, ref_index=0):
    buttons_keys=buttons_list[0].keys()
    united_button=dict.fromkeys(buttons_keys)
    for key in buttons_keys:
        if key=='args':
            all_annotations=[buttons[key][0]['scene.annotations'] for buttons in buttons_list]
            united_annotations=list(zip(*all_annotations))
            united_button[key]=[{'scene.annotations': united_annotations}]
        else:
            united_button[key]=buttons_list[ref_index][key]
    return united_button

def unite_updatemenus(updatemenus_list, ref_index=0):
    menus_keys=updatemenus_list[ref_index][0].keys()
    united_updatemenus_list=dict.fromkeys(menus_keys)
    for key in menus_keys:
        if key=='buttons':
            buttons_list=[updatemenus[0].get(key) for updatemenus in updatemenus_list]
            buttons_num=len(buttons_list[0])   
            segregated_buttons=[]
            for i in range(buttons_num):
                type_buttons=[buttons[i] for buttons in buttons_list]
                segregated_buttons.append(type_buttons)
            buttons=[unite_buttons(buttons) for buttons in segregated_buttons]
            united_updatemenus_list[key]=buttons
        else:
            united_updatemenus_list[key]=updatemenus_list[ref_index][0][key]
    return [united_updatemenus_list]


def compare_molecules(coordinates_df_list: List[pd.DataFrame],conformer_numbers:List[int]=None):
    if conformer_numbers is None:
        ## make a list of indices of conformers
        conformer_numbers= range(len(coordinates_df_list))
    # Create a subplot with 3D scatter plot
    colors_list=['red','magenta','purple','blue','green','yellow','orange','brown','black','pink','cyan']
    new_coodinates_df_list=[coordinates_df_list[i] for i in conformer_numbers]
    # new_coodinates_df_list=renumbering_df_list(new_coodinates_df_list)
    # coordinates_df_list=renumber_xyz_by_mcs(coordinates_df_list)  ##needs fixing , renumbering not working.
    xyz_df=(coordinates_df_list[conformer_numbers[0]])
    data_main, annotations_id_main, updatemenus = plot_interactions(xyz_df,'grey')
    updatemenus_list=[updatemenus]
    # Iterate through the conformer numbers and create traces for each conformer
    for  conformer_number,color in zip((conformer_numbers[1:]),colors_list):
        xyz_df = coordinates_df_list[conformer_number]
        data, annotations_id, updatemenus_main = plot_interactions(xyz_df,color)
        data_main += data
        annotations_id_main += annotations_id
        updatemenus_list.append(updatemenus_main)
    # Set axis parameters
    updatemenus=unite_updatemenus(updatemenus_list)
    axis_params = dict(showgrid=False, showbackground=False, showticklabels=False, zeroline=False,
                       titlefont=dict(color='white'))
    # Set layout
    layout = dict(scene=dict(xaxis=axis_params, yaxis=axis_params, zaxis=axis_params, annotations=annotations_id_main),
                  margin=dict(r=0, l=0, b=0, t=0), showlegend=False, updatemenus=updatemenus)
    fig = go.Figure(data=data_main, layout=layout)
    # fig.show()
    return fig

import dash
from dash import html, dcc, Output, Input, State
import plotly.graph_objects as go
import pandas as pd

def show_single_molecule(molecule_name,xyz_df=None, color='black'):
    if xyz_df is None:
        xyz_df=get_df_from_file(choose_filename()[0])

    data_main, annotations_id_main, updatemenus = plot_interactions(xyz_df,color)
    # Set axis parameters
    axis_params = dict(showgrid=False, showbackground=False, showticklabels=False, zeroline=False,
                       titlefont=dict(color='white'))
    # Set layout
    layout = dict(title=dict(text=molecule_name, x=0.5, y=0.9, xanchor='center', yanchor='top'),scene=dict(xaxis=axis_params, yaxis=axis_params, zaxis=axis_params, annotations=annotations_id_main),
                  margin=dict(r=0, l=0, b=0, t=0), showlegend=False, updatemenus=updatemenus)
    fig = go.Figure(data=data_main, layout=layout)
    html=fig.show()
    run_app(fig)

    
    return html


def run_app(figure):
        # Create a Dash app
    app = dash.Dash(__name__)

    # App layout
    app.layout = html.Div([
        dcc.Graph(id='molecule-plot', figure=figure), # Replace "Water" with your molecule
        html.Div(id='clicked-data', children=[]),
        html.Button('Save Clicked Atom', id='save-button', n_clicks=0),
        html.Div(id='saved-atoms', children=[])
    ])

    # Callback to display clicked data
    @app.callback(
        Output('clicked-data', 'children'),
        Input('molecule-plot', 'clickData'),
        prevent_initial_call=True
    )
    def display_click_data(clickData):
        if clickData:
            return f"Clicked Point: {clickData['points'][0]['pointIndex']}"
        return "Click on an atom."

    # Callback to save clicked atom index
    @app.callback(
        Output('saved-atoms', 'children'),
        Input('save-button', 'n_clicks'),
        State('molecule-plot', 'clickData'),
        State('saved-atoms', 'children'),
        prevent_initial_call=True
    )
    def save_clicked_atom(n_clicks, clickData, saved_atoms):
        if clickData:
            saved_atoms.append(clickData['points'][0]['pointIndex'])
        return f"Saved Atom Indices: {saved_atoms}"
        



if __name__ == '__main__':
    
    pass


In [35]:

def get_file_name_list(file_identifier):
    """
    The function gets a file identifier as input and returns a list of all files in the working 
    which contain the identifier in the files name
    ----------
    Parameters
    ----------
    identifier : str.
        The wanted file identifier like 'txt','info','nbo' contained in the filename
    -------
    Returns
    -------
    list
        A list of all files in the working directory with the chosen extension 
    --------
    Examples
    --------
    
    all_files_in_dir=listdir()
    print(all_files_in_dir)
        ['0_1106253-mod-mod.xyz', '0_1106253-mod.xyz', '1106253.cif', '1109098.cif', '1_1106253-mod.xyz', 'centered_0_BASCIH.xyz', 'cif_handler.py']
        
    xyz_files_in_dir=get_filename_list('.xyz')
    print(xyz_files_in_dir)
        ['0_1106253-mod-mod.xyz', '0_1106253-mod.xyz', '1_1106253-mod.xyz', 'centered_0_BASCIH.xyz']
  
    """
    return [filename for filename in os.listdir() if file_identifier in filename]

def split_strings(strings_list):
    split_list = []
    for string in strings_list:
        split_list.extend(string.split())
    return split_list

def get_df_from_file(filename,columns=['atom','x','y','z'],index=None):
    """
    Parameters
    ----------
    filename : str
        full file name to read.
    columns : str , optional
        list of column names for DataFrame. The default is None.
    splitter : str, optional
        input for [.split().] , for csv-',' for txt leave empty. The default is None.
    dtype : type, optional
        type of variables for dataframe. The default is None.

    Returns
    -------
    df : TYPE
        DESCRIPTION.

    """
    with open(filename, 'r') as f:
        lines=f.readlines()[2:]
    splitted_lines=split_strings(lines)
    df=pd.DataFrame(np.array(splitted_lines).reshape(-1,4),columns=columns,index=index)
    df[['x','y','z']]=df[['x','y','z']].astype(float)
    return df

In [36]:
from typing import List
import numpy.typing as npt
def adjust_indices(element):
    
    if isinstance(element, list):
        return [adjust_indices(sub_element) for sub_element in element]
    elif isinstance(element, int):
        return element - 1
    elif isinstance(element, np.ndarray):
        return element - 1
    elif isinstance(element, np.int64):
        return element - 1
    else:
        raise ValueError("Unsupported element type")




def calc_angle(p1: npt.ArrayLike, p2: npt.ArrayLike, degrees: bool=False) -> float: ###works, name in R: 'angle' , radians
    dot_product=np.dot(p1, p2)
    norm_p1=np.linalg.norm(p1)
    norm_p2=np.linalg.norm(p2)
    thetha=np.arccos(dot_product/(norm_p1*norm_p2))
    if degrees:
        thetha=np.degrees(thetha)   
    return thetha
    
def calc_new_base_atoms(coordinates_array: npt.ArrayLike, atom_indices: npt.ArrayLike):  #help function for calc_coordinates_transformation
    """
    a function that calculates the new base atoms for the transformation of the coordinates.
    optional: if the atom_indices[0] is list, compute the new origin as the middle of the first atoms.
    """
   
    if isinstance(atom_indices[0], list):
        new_origin=np.mean(coordinates_array[atom_indices[0]], axis=0)
    else:
        new_origin=coordinates_array[atom_indices[0]]
    new_y=(coordinates_array[atom_indices[1]]-new_origin)/np.linalg.norm((coordinates_array[atom_indices[1]]-new_origin))
    coplane=((coordinates_array[atom_indices[2]]-new_origin)/np.linalg.norm((coordinates_array[atom_indices[2]]-new_origin)+0.00000001))
    
    return (new_origin,new_y,coplane)

def np_cross_and_vstack(plane_1, plane_2):
    cross_plane=np.cross(plane_1, plane_2)
    united_results=np.vstack([plane_1, plane_2, cross_plane])
    return united_results

from numpy.typing import ArrayLike

def calc_basis_vector(origin, y: ArrayLike, coplane: ArrayLike):
    """
    Calculate the new basis vector.
    
    Parameters
    ----------
    origin : array-like
        The origin of the new basis.
    y : array-like
        The new basis's y direction.
    coplane : array-like
        A vector coplanar with the new y direction.
    
    Returns
    -------
    new_basis : np.array
        The computed new basis matrix.
    """
    coef_mat = np_cross_and_vstack(coplane, y)
    angle_new_y_coplane = calc_angle(coplane, y)
    cop_ang_x = angle_new_y_coplane - (np.pi/2)
    result_vector = [np.cos(cop_ang_x), 0, 0]
    new_x, _, _, _ = np.linalg.lstsq(coef_mat, result_vector, rcond=None)
    new_basis = np_cross_and_vstack(new_x, y)
    return new_basis

def transform_row(row_array, new_basis, new_origin, round_digits):
    """
    Transform a single row of coordinates.
    
    Parameters
    ----------
    row_array : array-like
        A row of coordinates.
    new_basis : array-like
        New basis matrix.
    new_origin : array-like
        The new origin.
    round_digits : int
        Number of decimal places to round to.
    
    Returns
    -------
    np.array
        The transformed coordinates.
    """
    row_array = np.squeeze(row_array)
    translocated_row = row_array - new_origin
    ## make sure or change to shape (3,) if needed
    if translocated_row.shape != (3,):
        try:
            translocated_row = np.reshape(translocated_row, (3,))
        except Exception as e:
            print("transform_row: Error reshaping translocated_row:", e)
    
    result = np.dot(new_basis, translocated_row).round(round_digits)
    return result

def calc_coordinates_transformation(coordinates_array: ArrayLike, base_atoms_indices: ArrayLike, round_digits: int = 4, origin: ArrayLike = None) -> ArrayLike:
    """
    Transform a coordinates array to a new basis defined by the atoms in base_atoms_indices.
    
    Parameters
    ----------
    coordinates_array : np.array
        The original xyz molecule coordinates.
    base_atoms_indices : list or array-like
        Indices of atoms to define the new basis.
    round_digits : int, optional
        Number of digits to round the result.
    origin : array-like, optional
        A new origin for the transformation. If None, the first base atom is used.
    
    Returns
    -------
    transformed_coordinates : np.array
        The transformed coordinates.
    """
    indices = adjust_indices(base_atoms_indices)
    # Calculate new basis using helper function calc_new_base_atoms (assumed to return origin, y, and coplane)
    new_basis = calc_basis_vector(*calc_new_base_atoms(coordinates_array, indices))
    if origin is None:
        new_origin = coordinates_array[indices[0]]
    else:
        new_origin = origin
    transformed_coordinates = np.apply_along_axis(lambda x: transform_row(x, new_basis, new_origin, round_digits), 1, coordinates_array)
  
    return transformed_coordinates

def preform_coordination_transformation(xyz_df, indices=None):
    """
    Perform a coordination transformation on the xyz DataFrame.
    
    Parameters
    ----------
    xyz_df : pd.DataFrame
        DataFrame containing columns 'x', 'y', 'z'.
    indices : array-like, optional
        Atom indices to use for the new basis. If None, default indices [1,2,3] are used.
    
    Returns
    -------
    xyz_copy : pd.DataFrame
        DataFrame with transformed coordinates.
    """
    xyz_copy = xyz_df.copy()
    coordinates = np.array(xyz_copy[['x', 'y', 'z']].values)
    if indices is None:
        transformed = calc_coordinates_transformation(coordinates, [1,2,3])
    else:
        transformed = calc_coordinates_transformation(coordinates, indices)
    
    xyz_copy[['x', 'y', 'z']] = transformed
    return xyz_copy


In [37]:
os.chdir(r'C:\Users\edens\Documents\GitHub\lucas_project\Secondary_Sphere\xyz_before\try')

In [38]:
df_list=[get_df_from_file(file) for file in get_file_name_list('.xyz')]
df_list[1]=preform_coordination_transformation(df_list[1],indices=[1,2,3])
df_list[2]=preform_coordination_transformation(df_list[2],indices=[6,7,8])
compare_molecules(df_list)

{frozenset({0, 1}): 1.37, frozenset({1, 2}): 0.96, frozenset({3, 4}): 0.96, frozenset({5, 6}): 1.4, frozenset({9, 10}): 1.39, frozenset({16, 15}): 1.09, frozenset({5, 7}): 1.41, frozenset({8, 6}): 1.39, frozenset({9, 7}): 1.39, frozenset({8, 10}): 1.39, frozenset({17, 15}): 1.09, frozenset({0, 3}): 1.37, frozenset({18, 15}): 1.09, frozenset({8, 12}): 1.08, frozenset({9, 13}): 1.08, frozenset({10, 14}): 1.08, frozenset({0, 5}): 1.57, frozenset({11, 6}): 1.08, frozenset({15, 7}): 1.51}
{frozenset({0, 1}): 1.39, frozenset({1, 2}): 1.38, frozenset({2, 3}): 1.39, frozenset({3, 4}): 1.39, frozenset({4, 5}): 1.39, frozenset({9, 10}): 1.2, frozenset({9, 12}): 1.11, frozenset({8, 4}): 1.08, frozenset({1, 11}): 1.08, frozenset({0, 5}): 1.39, frozenset({2, 7}): 1.08, frozenset({0, 6}): 1.08, frozenset({9, 3}): 1.48, frozenset({5, 13}): 1.73}
{frozenset({0, 1}): 1.39, frozenset({1, 2}): 1.38, frozenset({2, 3}): 1.39, frozenset({3, 4}): 1.39, frozenset({4, 5}): 1.39, frozenset({8, 9}): 1.2, frozens

In [39]:
df_list=[get_df_from_file(file) for file in get_file_name_list('.xyz')]
compare_molecules(df_list)

{frozenset({0, 1}): 1.37, frozenset({1, 2}): 0.96, frozenset({3, 4}): 0.96, frozenset({5, 6}): 1.4, frozenset({9, 10}): 1.39, frozenset({16, 15}): 1.09, frozenset({5, 7}): 1.41, frozenset({8, 6}): 1.39, frozenset({9, 7}): 1.39, frozenset({8, 10}): 1.39, frozenset({17, 15}): 1.09, frozenset({0, 3}): 1.37, frozenset({18, 15}): 1.09, frozenset({8, 12}): 1.08, frozenset({9, 13}): 1.08, frozenset({10, 14}): 1.08, frozenset({0, 5}): 1.57, frozenset({11, 6}): 1.08, frozenset({15, 7}): 1.51}
{frozenset({0, 1}): 1.39, frozenset({1, 2}): 1.38, frozenset({2, 3}): 1.39, frozenset({3, 4}): 1.39, frozenset({4, 5}): 1.39, frozenset({9, 10}): 1.2, frozenset({9, 12}): 1.11, frozenset({8, 4}): 1.08, frozenset({1, 11}): 1.08, frozenset({0, 5}): 1.39, frozenset({2, 7}): 1.08, frozenset({0, 6}): 1.08, frozenset({9, 3}): 1.48, frozenset({5, 13}): 1.73}
{frozenset({0, 1}): 1.39, frozenset({1, 2}): 1.38, frozenset({2, 3}): 1.39, frozenset({3, 4}): 1.39, frozenset({4, 5}): 1.39, frozenset({8, 9}): 1.2, frozens